# 2D Shallow Neural Network Position-Velocity Model

Note the *2D* part refers to the ball is being shot from a point to different points in the goal, **not** that the input features are 2D. In fact, the input features are 4D `[x, y, dx, dy]` vectors

### Load Data

In [6]:
%store -r many_many_long

sideways = many_many_long.T

X, Y = np.array(sideways[:4]).astype(float), np.array(sideways.ix[4, :]).astype(int)

### Train

In [ ]:
from nn.shallow.model import NeuralNetwork

nn = NeuralNetwork(X, Y, H=10, C=5, learning_rate=.1, regularizer=0, gradient_checking=False, inspect=True)

def states(iters):
    for _ in range(iters):
        nn.learn()
        yield nn.info
        
states = list(states(50000))

### Plot Losses

In [ ]:
def loss_averages():
    for state in states:
        yield state.loss.mean()
        
loss_avgs = list(loss_averages())

pd.DataFrame(loss_avgs, columns=['loss']).plot()

### Visualize

Notice the plots are linear because of the following recurrence relation:

$$v_t = x_{t+1} - x_t$$

This formula follows by definition of $x_t$:

$$x_t = x_{t-1} + v_{t-1}$$

Intuitively, as the difference between $x$ points gets smaller, the difference between $y$ points get smaller **by the same amount**

In [ ]:
from ipywidgets import interact, fixed

@interact(X=fixed(X), Y=fixed(Y), many_many_long=fixed(many_many_long), view_index=(0, len(states)-1), states=fixed(states))
def plot(X, Y, many_many_long, view_index, states):
    """View decision boundary over time
    
    We have the state of the model (i.e. its parameters) after every time step.
    We want to see how the model learns over time. Hence create a one-off instance
    of a softmax with its old parameters and ask it to make a bunch of predictions!
    """
    s = states[view_index]
    nn = NeuralNetwork(X=X, ys_train=Y, H=10, C=5, Wh=s.Wh, bh=s.bh, Ws=s.Ws, bs=s.bs)
    
    # Plotting
    colormap = np.array(['r', 'g', 'b', 'y', 'violet'])
    
    # Plot the gold standard in a plot right next to it
    many_many_long.plot(kind='scatter', x='x', y='y', c=colormap[many_many_long['label']], title='Ground Truth')

    # Predictions
    many_many_long['predictions'] = nn.predict(X)
    many_many_long.plot(kind='scatter', x='x', y='y', c=colormap[many_many_long['predictions']], title='Predictions')
    
    # Misclassified examples
    many_many_long['correct'] = many_many_long['label'] == many_many_long['predictions']
    many_many_long.plot(kind='scatter', x='x', y='y', c=colormap[many_many_long['correct']], title='Missclassified')

In [ ]:
many_many_long['correct'].sum() / len(many_many_long)